<a href="https://colab.research.google.com/github/josepmatias/nos_reten-ao/blob/main/cadastro_retencao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importar módulos
import pandas as pd

In [ ]:
!pip install pyxlsb

In [ ]:
#designar nomes de ficheiros e designar o mês
cadastro_nome="Cadastro_EADW.xlsb"
agentes_nome="AGENTES_TODOS_ENTIDADE_GESTORA_KC13219_29.04.xlsx"
cav_sfid_nome="SFIDsvsCAVs_20240506124348.xlsb"

mes="202404"

In [ ]:
#fazer upload dos ficheiros
from google.colab import files
uploaded = files.upload()
uploaded2=files.upload()
uploaded3=files.upload()

In [ ]:
#importar cadastro
cadastro=pd.read_excel(cadastro_nome, sheet_name=0)
cadastro2=cadastro.copy()
cadastro2.columns=cadastro.iloc[0, :]
cadastro2.columns

In [ ]:
#importar ficheiro "agentes todos entidade gestora" do mês

agentes=pd.read_excel(agentes_nome, sheet_name=0)

In [ ]:
agentes.columns

In [ ]:
#importar ficheiro sidsvscavs
sfids_cavs=pd.read_excel(cav_sfid_nome, sheet_name=0)

In [ ]:
#formatar colunas na dataframe sfids_cavs

sfids_cavs.columns=[i.strip() for i in sfids_cavs.columns]

In [ ]:
#filtrar dataframe para os dados do mês n

cadastro_mes=cadastro2[cadastro2.iloc[:, 0]==mes]


In [ ]:
#criação colunas inbound e outbound - chaves representam colunas a criar, valores representam colunas no ficheiro entidade agente

def conversao (dici, str, df=cadastro_mes, df2=agentes):
  for i, j in dici.items():
    for cav in df["CAV " + str].unique().tolist():
      indice=df.loc[df["CAV " + str]==cav, :].index
      var=df2.loc[df2.iloc[:, 0]==cav, j].unique().tolist()[0]
      df.loc[indice, i]=var

  return df

#dicionario inb
dici_inb={"Delegação IN_EntidadeGestora":"ID Delegação do Agente", "Entidade (EG)": "ID Entidade do Agente",  "Tipo do Agente": "Tipo do Agente", "Rede (EG)- Inbound": "Rede do Agente",\
"Estado cav inbound": "Situação do Agente"}

conversao(dici_inb, "INBOUND")

#dicionario out

dici_out={"Delegação OUT_EntidadeGestora":"ID Delegação do Agente", "Rede (EG)- Outbound":"Rede do Agente", "Estado Cav Outbound": "Situação do Agente"}


conversao(dici_out, "OUTBOUND")


In [ ]:
#criar colunas "CHAVE INB" e "CHAVE OUT" na dataframe cadastro_mes

for i, j, k in zip(["CAV INBOUND", "CAV OUTBOUND"], ["Delegação IN_EntidadeGestora", "Delegação OUT_EntidadeGestora"], ["COD ENTIDADE", "COD ENTIDADE"]):
  if i.find("OUTBOUND")!=-1:
    cadastro_mes["CHAVE OUT"]=cadastro_mes[i] + cadastro_mes[j] + cadastro_mes[k]
  else:
    cadastro_mes["CHAVE INB"]=cadastro_mes[i] + cadastro_mes[j] + cadastro_mes[k]




In [ ]:
#chave entidade, delegação, cav
sfids_cavs["Chave (del_ent_cav)"]=sfids_cavs["CAV Entity"].astype("str") + sfids_cavs["CAV Delegation"].astype("str") + sfids_cavs["CAV"].astype("str")

#chave cav, delegação, entidade

sfids_cavs["Chave (cav_ent_del)"]=sfids_cavs["CAV"].astype("str") + sfids_cavs["CAV Delegation"].astype("str") + sfids_cavs["CAV Entity"].astype("str")

In [ ]:
sfids_cavs.columns

In [ ]:
#construir coluna "sfid_in" e "sfid_out"

for i, j in zip(["SFID IN", "SFID OUT"], ["CHAVE INB", "CHAVE OUT"]):
  print(f"{i} -> {j}")
  chaves=cadastro_mes[j].unique().tolist()
  for chave in chaves:
    indice=cadastro_mes.loc[cadastro_mes[j]==chave, :].index
    try:
      sfid=sfids_cavs.loc[sfids_cavs["Chave (cav_ent_del)"]==chave, "Dealer SFID"].tolist() [0]
      cadastro_mes.loc[indice, i]=sfid
    except:
      cadastro_mes.loc[indice, i]="#N/A"





In [ ]:
#criar colunas "delegação t/f" e "entidade t/f"

#def deleg (df, col1, col2):

cadastro_mes["Delegação t/f"]=cadastro_mes['Delegação IN_EntidadeGestora']==cadastro_mes['Delegação OUT_EntidadeGestora']


cadastro["Entidade t/f"]=cadastro_mes["COD ENTIDADE"]==cadastro_mes["Entidade (EG)"]


In [ ]:
cadastro_mes[['Delegação IN_EntidadeGestora', 'Delegação OUT_EntidadeGestora']].head()

In [ ]:
cadastro_mes.columns